In [2]:
import requests
import json
import time
import sqlite3
import pandas as pd

# 0. Consulta

In [3]:
registro_i = requests.get("https://www.boliviasegura.gob.bo/wp-content/json/api.php")

In [4]:
registro_i_json = registro_i.json()
# Elementos en fecha de entrada
# ['fecha', 'departamento', 'contador', 'total', 'porcentaje']

In [66]:
#with open("20200424.json","r") as json_arch : 
#    registro_i_json = json.load(json_arch)

In [5]:
registro_i_json["fecha"] # retroceder un día

'24/04/20 20:59'

In [6]:
with open('{}.json'.format(pd.to_datetime(registro_i_json["fecha"]).strftime("%Y%m%d")), 'w') as outfile:
    json.dump(registro_i_json, outfile)

## 1. Parseo del imput para cifras acumuladas

### Por departamentos

In [7]:
day_i = {}
for key, value in registro_i_json["departamento"].items()  :
    val_j =  value["contador"]
    val_j["total"]  = value["total"] 
    day_i[key] = val_j

df_day_i = pd.DataFrame(day_i).T.rename_axis(["depto"]).reset_index()
df_day_i["fecha"] = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
df_day_i["depto"] = df_day_i["depto"].str.upper()
df_day_i["activos"] = df_day_i.confirmados - df_day_i.decesos - df_day_i.recuperados
df_day_i

,depto,confirmados,decesos,recuperados,sospechosos,descartados,total,fecha,activos
0,LP,155,14,13,4,1168,1354,2020-04-24,128
1,CB,70,6,13,0,900,989,2020-04-24,51
2,SC,446,19,14,0,1786,2265,2020-04-24,413
3,OR,65,2,8,0,258,333,2020-04-24,55
4,PT,21,1,0,11,103,136,2020-04-24,20
5,TJ,3,0,0,7,36,46,2020-04-24,3
6,CH,2,0,0,3,115,120,2020-04-24,2
7,BN,33,1,0,40,53,127,2020-04-24,32
8,PN,12,1,6,2,71,92,2020-04-24,5


### A nivel nacional

In [8]:
bol_i = registro_i_json["contador"]
bol_i["total"] = registro_i_json["total"]

sr_bol_i = pd.Series(bol_i)
sr_bol_i["fecha"] = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
sr_bol_i["depto"] = "BOL"
sr_bol_i["activos"] = sr_bol_i.confirmados - sr_bol_i.decesos - sr_bol_i.recuperados
sr_bol_i


confirmados           807
decesos                44
recuperados            54
sospechosos            67
descartados          4490
total                5462
fecha          2020-04-24
depto                 BOL
activos               709
dtype: object

# 2. Calculando variaciones

In [9]:
valores = [j.to_list() for i,j  in df_day_i.iterrows()]


In [15]:
df_day_i.columns

Index(['depto', 'confirmados', 'decesos', 'recuperados', 'sospechosos',
       'descartados', 'total', 'fecha', 'activos'],
      dtype='object')

In [16]:
conn = sqlite3.connect('BD_COVID19_BOL.sqlite')
cur = conn.cursor()

valores = [j.to_list() for i,j  in df_day_i[['depto', 'confirmados', 'decesos', 'recuperados', 'sospechosos',
       'descartados', 'total', 'fecha', 'activos']].iterrows()]
cur.executemany("""INSERT INTO daily_covid19_BO(daily_depto
,daily_total_confirmados
,daily_total_decesos
,daily_total_recuperados
,daily_total_sospechosos
,daily_total_descartados
,daily_total_total
,daily_fecha
,daily_total_activos) VALUES (?,?,?,?,?,?,?,?,?);""", valores)
conn.commit()
conn.close()


### Extrayendo el acumulado del día anterior

In [12]:
# Extraer de base de datos los elememtos del día anterior para obtener las variaciones

In [17]:
fecha_actualizacion = pd.to_datetime(registro_i_json["fecha"]).strftime("%Y-%m-%d")
fecha_actualizacion

'2020-04-24'

In [114]:
fecha_actualizacion = '2020-04-21'

In [18]:
conn = sqlite3.connect('BD_COVID19_BOL.sqlite')

df_t_1 = pd.read_sql_query("""select * 
from daily_covid19_BO
where daily_fecha = strftime('%Y-%m-%d',date('{fecha_consulta}', '-1 day'))
order by daily_depto """.format(fecha_consulta = fecha_actualizacion), conn)

df_t = pd.read_sql_query("""select * 
from daily_covid19_BO
where daily_fecha = '{fecha_consulta}'
order by daily_depto """.format(fecha_consulta = fecha_actualizacion), conn)


conn.close()


delta = df_t[[  'daily_total_confirmados', 'daily_total_activos',
       'daily_total_decesos', 'daily_total_recuperados',
       'daily_total_sospechosos', 'daily_total_descartados',
       'daily_total_total']].values -   df_t_1[[  'daily_total_confirmados', 'daily_total_activos',
       'daily_total_decesos', 'daily_total_recuperados',
       'daily_total_sospechosos', 'daily_total_descartados',
       'daily_total_total']].values

delta_df = pd.DataFrame(delta , columns=['daily_delta_confirmados',
'daily_delta_activos', 'daily_delta_decesos', 'daily_delta_recuperados',
'daily_delta_sospechosos', 'daily_delta_descartados',
'daily_delta_total'])

delta_df[['daily_id', 'daily_fecha', 'daily_depto']] = df_t[['daily_id', 'daily_fecha', 'daily_depto']]

valores_delta = [j.to_list() for i,j  in delta_df.iterrows()]

conn = sqlite3.connect('BD_COVID19_BOL.sqlite')
cur = conn.cursor()

for val_i in valores_delta : 
    cur.execute("""UPDATE  daily_covid19_BO
    SET   daily_delta_confirmados = {0}
    , daily_delta_activos     =     {1} 
    , daily_delta_decesos     =     {2}
    , daily_delta_recuperados =  {3}
    , daily_delta_sospechosos =   {4}
    , daily_delta_descartados =   {5}
    , daily_delta_total       =   {6}                   
    where daily_id = {7}
    and daily_fecha = '{8}'
    and daily_depto = '{9}'  """.format(*val_i))
conn.commit()
conn.close()

ValueError: operands could not be broadcast together with shapes (18,7) (9,7) 